### Assigment 4

**Submission deadlines**:

* get at least 4 points by 17.05.2023
* remaining points: last lab session before or on 26.05.2023

**Points:** Aim to get 12 out of 15+ possible points

All needed data files should be on Drive: <https://drive.google.com/drive/folders/1HaMbhzaBxxNa_z_QJXSDCbv5VddmhVVZ?usp=sharing> 



In [3]:
!ls

'Assigment 4-20230526T031220Z-001.zip'	 task3_polish_lower.txt.gz
 sample_data				 task3_polish_upper.txt.gz


## Task 1 (5 points)

Implement simplified word2vec with negative sampling from scratch (using pure numpy). Assume that in the training data objects and contexts are given explicitly, one pair per line, and objects are on the left. The result of the training should be object vectors. Please, write them to a file using *natural* text format, ie

<pre>
word1 x1_1 x1_2 ... x1_N 
word2 x2_1 x2_2 ... x2_N
...
wordK xK_1 xK_2 ... xk_N
</pre>

Use the loss from Slide 25 in Lecture NLP.01, compute the gradient manually. You can use some gradient clipping, or regularisation. 

**Remark**: the data is specially prepared to make the learning process easier. 
Present vectors using the code below. In this task we define success as 'obtaining a result which looks definitely not random'

In [ ]:
from gensim.models import KeyedVectors
task1_wv = KeyedVectors.load_word2vec_format('task1_w2v_vectors.txt', binary=False)

example_english_words = ['dog', 'dragon', 'love', 'bicycle', 'marathon', 'logic', 'butterfly']  # replace, or add your own examples
example_polish_words = ['pies', 'smok', 'miłość', 'rower', 'maraton', 'logika', 'motyl']

example_words = example_polish_words

for w0 in example_words:
    print ('WORD:', w)
    for w, v in task1_wv.most_similar(w0):
        print ('   ', w, v)
    print ()

## Task 2 (4 points)

Your task is to train the embeddings for Simple Wikipedia titles, using gensim library. As the example below shows, training is really simple:

```python
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
```
*sentences* can be a list of list of tokens, you can also use *gensim.models.word2vec.LineSentence(source)* to create restartable iterator from file. At first, use [this file] containing such pairs of titles, that one article links to another.

We say that two titles are *related* if they both contain a word (or a word bigram) which is not very popular (it occurs only in several titles). Make this definition more precise, and create the corpora which contains pairs of related titles. Make a mixture of the original corpora, and the new one, then train title vectors again.

Compare these two approaches using similar code to the code from Task 1.

In [ ]:
# The cell for your presentation

# Task 3 (4 points)

Suppose that we have two languages: Upper and Lower. This is an example Upper sentence:

<pre>
THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG.
</pre>

And this is its translation into Lower:

<pre>
the quick brown fox jumps over the lazy dog
</pre>

You have two corpora for these languages (with different sentences). Your task is to train word embedings for both languages together, so as to make embeddings of the words which are its translations as close as possible. But unfortunately, you have the budget which allows you to prepare the translation only for 1000 words (we call it D, you have to deside which words you want to be in D)

Prepare the corpora wich contains three kind of sentences:
* Upper corpus sentences
* Lower corpus sentences
* sentences derived from Upper/Lower corpus, modified using D

There are many possible ways of doing this, for instance this one (ROT13.COM: hfr rirel fragrapr sebz obgu pbecben gjvpr: jvgubhg nal zbqvsvpngvbaf, naq jvgu rirel jbeqf sebz Q ercynprq ol vgf genafyngvba)

We define the score for an Upper WORD as  $\frac{1}{p}$, where $p$ is a position of its translation in the list of **Lower** words most similar to WORD. For instance, when most similar words to DOG are:

<pre>
WOLF, CAT, WOLVES, LION, gopher, dog
</pre>

then the score for the word DOG is 0.5. Compute the average score separately for words from D, and for words out of D (hint: if the computation takes to much time do it for a random sample).


In [4]:
file_lower = "task3_polish_lower.txt.gz"
file_upper = "task3_polish_upper.txt.gz"

In [5]:
!gzip -d task3_polish_lower.txt.gz && gzip -d task3_polish_upper.txt.gz

In [6]:
!ls

'Assigment 4-20230526T031220Z-001.zip'	 task3_polish_lower.txt
 sample_data				 task3_polish_upper.txt


In [8]:
from collections import Counter
from random import sample
from gensim.models import Word2Vec
from tqdm import tqdm

### Reading the data

In [19]:
%%time
upper_words = set()
all_line_words = []
corpus = []

with open("task3_polish_lower.txt", "r") as f:
    for line in f.readlines():
        line_words = line.split()
        all_line_words += line_words

with open("task3_polish_upper.txt", "r") as f:
    for line in f.readlines():
        line_words = line.split()
        for word in line_words:
            upper_words.add(word)

        corpus.append(line_words)
        line_words = [word.lower() for word in line_words]
        all_line_words += line_words

all_line_words = Counter(all_line_words)

CPU times: user 22.9 s, sys: 1.93 s, total: 24.8 s
Wall time: 25.8 s


In [20]:
len(all_line_words)

29877

In [21]:
%%time
D_size = 1000
min_word_length = 4
top_words = set()

for word, word_count in all_line_words.most_common():
    if len(word) >= min_word_length:
        top_words.add(word.lower())
        
    if len(top_words) >= D_size:
        break

CPU times: user 26.4 ms, sys: 1e+03 µs, total: 27.4 ms
Wall time: 31.5 ms


In [22]:
D_lower, D_upper = {}, {}

for w in top_words:
    wu = w.upper()
    wl = w.lower()
    D_lower[wu] = wl
    D_upper[wl] = wu

In [23]:
list(D_upper.items())[:6], list(D_lower.items())[:6]

([('sektor', 'SEKTOR'),
  ('izba', 'IZBA'),
  ('czynność', 'CZYNNOŚĆ'),
  ('obowiązek', 'OBOWIĄZEK'),
  ('ubezpieczenie', 'UBEZPIECZENIE'),
  ('dobro_dobrze', 'DOBRO_DOBRZE')],
 [('SEKTOR', 'sektor'),
  ('IZBA', 'izba'),
  ('CZYNNOŚĆ', 'czynność'),
  ('OBOWIĄZEK', 'obowiązek'),
  ('UBEZPIECZENIE', 'ubezpieczenie'),
  ('DOBRO_DOBRZE', 'dobro_dobrze')])

In [24]:
%%time
with open("task3_polish_lower.txt", "r") as f:
    for line in f.readlines():
        line_words = line.split()
        line_words = [D_upper.get(word, word) for word in line_words]
        corpus.append(line_words)

with open("task3_polish_upper.txt", "r") as f:
    for line in f.readlines():
        line_words = line.split()
        line_words = [D_lower.get(word, word) for word in line_words]
        corpus.append(line_words)

CPU times: user 13.3 s, sys: 141 ms, total: 13.5 s
Wall time: 13.8 s


In [15]:
task3_model = Word2Vec(
    sentences=corpus, 
    vector_size=100, 
    window=5, 
    min_count=1, 
    workers=4, 
    sg=1
)

In [25]:
task3_model.wv.most_similar("SEKTOR", topn=5)

[('PRZEDSIĘBIORSTWO', 0.8303931355476379),
 ('branża', 0.7982797026634216),
 ('mśp', 0.7803087830543518),
 ('rolno-spożywczego', 0.7764489054679871),
 ('inwestujący', 0.7649809718132019)]

### Score words in D

In [33]:
def get_similar_words(fetch_topn: int, word: str) -> list[str]:
    return [
        result[0] 
        for result in task3_model.wv.most_similar(word, topn=fetch_topn)
    ]

In [35]:
%%time

score_in_D = 0
fetch_topn = 50

for w in top_words:
    similar_words = get_similar_words(word=w.upper(), fetch_topn=fetch_topn)
    for i, word in enumerate(similar_words):
        if word == word.lower():
            score_in_D += 1.0 / (i + 1)
            break

print(f"{score_in_D / 10 :.2f}%")

78.24%
CPU times: user 6.11 s, sys: 1.41 s, total: 7.52 s
Wall time: 5.78 s


### Score words not in D

In [46]:
%%time
score_not_in_D = 0
fetch_topn = 800
n_samples = 10000

words_not_in_d = [
    word for word in upper_words if word.lower() not in top_words
]

words_sample = sample(words_not_in_d, n_samples)
for w in tqdm(words_sample):
    similar_words = get_similar_words(word=w.upper(), fetch_topn=fetch_topn)
    for i, word in enumerate(similar_words):
        if word == word.lower():
            score_not_in_D += 1.0 / (i + 1)
            break


100%|██████████| 10000/10000 [01:23<00:00, 119.51it/s]

CPU times: user 1min 36s, sys: 38.5 s, total: 2min 14s
Wall time: 1min 23s


In [48]:
print(f"{score_not_in_D / n_samples * 100 :.2f}%")

14.24%


# Task 4 (4 points)

In this task you are asked to do two things:
1. compare the embeddings computed on small corpus (like Brown Corpus , see: <https://en.wikipedia.org/wiki/Brown_Corpus>) with the ones coming from Google News Corpus
2. Try to use other resourses like WordNet to enrich to corpus, and obtain better embeddings

You can use the following code snippets:

```python
# printing tokenized Brown Corpora
from nltk.corpus import brown
for s in brown.sents():
    print(*s)
    
#iterating over all synsets in WordNet
from nltk.corpus import wordnet as wn

for synset_type in 'avrns': # n == noun, v == verb, ...
    for synset in list(wn.all_synsets(synset_type)))[:10]:
        print (synset.definition())
        print (synset.examples())
        print ([lem.name() for lem in synset.lemmas()])
        print (synset.hyperonims()) # nodes 1 level up in ontology
        
# loading model and compute cosine similarity between words

model = Word2Vec.load('models/w2v.wordnet5.model') 
print (model.wv.similarity('dog', 'cat'))
```

Embeddings will be tested using WordSim-353 dataset, the code showing the quality is in the cell below. Prepare the following corpora:
1. Tokenized Brown Corpora
2. Definitions and examples from Princeton WordNet
3. (1) and (2) together
4. (3) enriched with pseudosentences containing (a subset) of WordNet knowledge (such as 'tiger is a carnivore')

Train 4 Word2Vec models, and raport Spearman correletion between similarities based on your vectors, and similarities based on human judgements.



In [ ]:
# Code for computing correlation between W2V similarity, and human judgements

import gensim.downloader
from scipy.stats import spearmanr

gn = gensim.downloader.load('word2vec-google-news-300')

for similarity_type in ['relatedness', 'similarity']:
    ws353 = []
    for x in open(f'wordsim_{similarity_type}_goldstandard.txt'): 
        a,b,val = x.split()
        val = float(val)
        ws353.append( (a,b,val))
    # spearmanr returns 2 vallues: correlation and pval. pval should be close to zero
    print (similarity_type + ':', spearmanr(vals, ys)) 

In [1]:
import nltk
import shutil
import string
import gensim
import gensim.downloader

from tqdm import tqdm

from nltk.corpus import wordnet as wn
from nltk.corpus import brown

from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity 

In [2]:
%%time
nltk.download('brown')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


CPU times: user 186 ms, sys: 124 ms, total: 310 ms
Wall time: 704 ms


True

### Brown vs Google News

In [3]:
!gdown "0B7XkCwpI5KDYNlNUTTlSS21pQmM"

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
100% 1.65G/1.65G [00:07<00:00, 234MB/s]


In [4]:
google_news = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin.gz', binary=True
)

In [6]:
%%time
# remove punctuation and create brown corpus

brown_corpus = []
for sent in brown.sents():
    sent =  ' '.join(sent)
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    sent = sent.split()
    sent = [i.lower() for i in sent]
    brown_corpus.append(sent)

CPU times: user 3.81 s, sys: 99.6 ms, total: 3.91 s
Wall time: 4.44 s


In [9]:
%%time
brown_corpus_model = Word2Vec(
    sentences=brown_corpus, 
    vector_size=100, 
    window=5, 
    min_count=1, 
    workers=4, 
    sg=1
)

CPU times: user 47.9 s, sys: 125 ms, total: 48 s
Wall time: 30.1 s


In [10]:
!ls

GoogleNews-vectors-negative300.bin.gz
sample_data
task4_wordsim_relatedness_goldstandard.txt
task4_wordsim_similarity_goldstandard.txt


In [26]:
# compare models using goldstandard

def evaluate(model):
    for t in (
        "task4_wordsim_relatedness_goldstandard.txt",
        "task4_wordsim_similarity_goldstandard.txt"
    ):
        similarities_vectors = []
        similarities_human = []
        with open(t) as f:
            for line in f.readlines():
                line_split = line.split()
                x = line_split[0].lower()
                y = line_split[1].lower()
                v = float(line_split[2])

                if all((x in model, y in model)):
                    similarities_human.append(v)
                    similarities_vectors.append(
                        cosine_similarity(
                            model[x].reshape(1,-1), 
                            model[y].reshape(1,-1)
                        )[0,0]
                    )
        score = spearmanr(similarities_vectors, similarities_human)
        print(f"Result {t[6:-5]} score: {score}")
                

In [27]:
evaluate(model=brown_corpus_model.wv)

Result wordsim_relatedness_goldstandar score: SignificanceResult(statistic=0.15864483117650194, pvalue=0.015352496557146414)
Result wordsim_similarity_goldstandar score: SignificanceResult(statistic=0.2959432553861848, pvalue=3.9169315228460296e-05)


In [29]:
evaluate(model=google_news)

Result wordsim_relatedness_goldstandar score: SignificanceResult(statistic=0.6219276471962092, pvalue=3.7240687603511443e-28)
Result wordsim_similarity_goldstandar score: SignificanceResult(statistic=0.7771097696126599, pvalue=4.360432258522682e-42)


### Enriched with pseudosentences from WordNet

In [31]:
%%time
wn_corpus = []
for synset_type in tqdm('avrns'): # n == noun, v == verb, ...
    for synset in list(wn.all_synsets(synset_type)):
        defintion = synset.definition()
        defintion = defintion.translate(
            str.maketrans('', '', string.punctuation)
        )
        defintion = defintion.split()
        defintion = [i.lower() for i in defintion]
        wn_corpus.append(defintion)
        for example in synset.examples():
            example = example.translate(
                str.maketrans('', '', string.punctuation)
            )
            example = example.split()
            example = [i.lower() for i in example]
            wn_corpus.append(example)

100%|██████████| 5/5 [00:06<00:00,  1.28s/it]

CPU times: user 5.92 s, sys: 471 ms, total: 6.39 s
Wall time: 6.4 s


In [32]:
%%time
wordnet_model = Word2Vec(
    sentences=wn_corpus, 
    vector_size=100, 
    window=5, 
    min_count=1, 
    workers=8, 
    sg=1
)

CPU times: user 1min 1s, sys: 239 ms, total: 1min 2s
Wall time: 41.2 s


In [33]:
evaluate(model=wordnet_model.wv)

Result wordsim_relatedness_goldstandar score: SignificanceResult(statistic=0.2598249075571557, pvalue=3.312448690865059e-05)
Result wordsim_similarity_goldstandar score: SignificanceResult(statistic=0.43692677652125295, pvalue=1.103805652074943e-10)


### WordNet + Brown

In [34]:
%%time
wn_brown_corpus = brown_corpus + wn_corpus

wordnet_brown_model = Word2Vec(
    sentences=wn_brown_corpus, 
    vector_size=100, 
    window=5, 
    min_count=1, 
    workers=8, 
    sg=1
)

CPU times: user 1min 57s, sys: 389 ms, total: 1min 57s
Wall time: 1min 13s


In [35]:
evaluate(model=wordnet_brown_model.wv)

Result wordsim_relatedness_goldstandar score: SignificanceResult(statistic=0.2719258321709995, pvalue=1.2468971184402208e-05)
Result wordsim_similarity_goldstandar score: SignificanceResult(statistic=0.4849921853813924, pvalue=2.583534627470877e-13)


### WordNet + Brown + Fake sentences from WordNet

In [36]:
%%time
wn_corpus_with_fake = []
for synset_type in tqdm('avrns'): # n == noun, v == verb, ...
    for synset in list(wn.all_synsets(synset_type)):
        defintion = synset.definition()
        defintion = defintion.translate(
            str.maketrans('', '', string.punctuation)
        )
        defintion = defintion.split()
        defintion = [i.lower() for i in defintion]
        wn_corpus_with_fake.append(defintion)
        for example in synset.examples():
            example = example.translate(
                str.maketrans('', '', string.punctuation)
            )
            example = example.split()
            example = [i.lower() for i in example]
            wn_corpus_with_fake.append(example)
        
        # fakes
        if len(synset.hypernyms()) > 0:
            lem = [lem.name() for lem in synset.lemmas()][0]
            hypernyms = [lem.name()[:-5] for lem in synset.hypernyms()]
            for hyper in hypernyms:
                wn_corpus_with_fake.append(lem + " is " + hyper)

100%|██████████| 5/5 [00:03<00:00,  1.36it/s]

CPU times: user 3.42 s, sys: 222 ms, total: 3.64 s
Wall time: 3.68 s


In [37]:
%%time
wn_fake_brown_corpus = brown_corpus + wn_corpus_with_fake

wordnet_fake_brown_model = Word2Vec(
    sentences=wn_fake_brown_corpus, 
    vector_size=100, 
    window=5, 
    min_count=1, 
    workers=8, 
    sg=1
)

CPU times: user 2min 18s, sys: 571 ms, total: 2min 18s
Wall time: 1min 23s


In [38]:
evaluate(model=wordnet_fake_brown_model.wv)

Result wordsim_relatedness_goldstandar score: SignificanceResult(statistic=0.27356162177594645, pvalue=1.0998403274931192e-05)
Result wordsim_similarity_goldstandar score: SignificanceResult(statistic=0.4883773246948573, pvalue=1.6664606694279177e-13)


# Task 5 (4 points)

Do the Problem 1 from old version of [Assigment 4](https://github.com/rnoxy/dl_uwr/blob/summer2023/Assignments/Assignment4.ipynb)

https://github.com/TheFebrin/UNIVERSITY/blob/master/Deep%20Learning/Assignments/Assignment5/Assignment5.ipynb